In [ ]:
!pip install faker -q
import random
from faker import Faker
import os
from google.colab import drive
from faker.providers.credit_card import CreditCard
import datetime
from datetime import timedelta

# Mount Google Drive to access data files
drive.mount('/content/gdrive')

# Define the folder path containing the files
folder_path = "/content/gdrive/My Drive/CSE341"

fake = Faker()

num_customers = 10
num_hotels = 5
num_amenities = 8
num_room_types = 8
num_reservations = 10
num_payments = 10
num_rooms = 15
num_managers = 5

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def create_customer(i):
    first_name = fake.first_name()
    last_name = fake.last_name()
    phone_number = f'+1({fake.random_number(digits=3):03d})-{fake.random_number(digits=3):03d}-{fake.random_number(digits=4):04d}'
    address_id = i
    membership_id = i
    card_id = i
    return first_name, last_name, phone_number, card_id, address_id, membership_id

def create_address(i):
    return (fake.street_address(), fake.city(), fake.state_abbr(), fake.zipcode())

def create_membership(i):
    return (random.randint(0, 10000))

def create_credit_card(i):
    card_types = ["visa", "mastercard", "amex", "discover"]
    card_type = random.choice(card_types)
    expiration_date = fake.date_between(start_date='+1y', end_date='+5y')
    expiration_date_str = expiration_date.strftime('%Y-%m-%d')
    return fake.credit_card_number(card_type=card_type), card_type.capitalize(), expiration_date_str

def create_room(i):
    return (random.randint(1, num_hotels), random.randint(1, 100), random.randint(1, num_room_types), random.choice(create_room_status()))

def create_hotel(i):
    return (fake.company(), i)

def create_amenity(i, hotel_id):
    amenities = ['Free Wi-Fi', 'Parking', 'Pool', 'Gym', 'Breakfast', 'Bar', 'Conference Room', 'Pet-friendly', 'Laundry']
    return i, amenities[i % len(amenities)], hotel_id

def create_room_type(i):
    room_type = ['Single', 'Double', 'Triple', 'Quad', 'Queen', 'King', 'Twin', 'Suite']
    return (room_type[min(i - 1, len(room_type) - 1)], 10)

def create_room_status():
    return ['Available', 'Occupied', 'Out of service']

def create_reservation_status():
    return ['Confirmed', 'Checked In', 'Checked Out', 'Cancelled']

def create_reservation(i):
    arrival_date = fake.date_between(start_date='-1y', end_date='+1y')
    
    # Use timedelta to ensure departure_date is always greater than arrival_date
    departure_date = arrival_date + timedelta(days=random.randint(1, 30))
    
    today = datetime.date.today()

    if arrival_date > today:
        status = "Confirmed"
    elif departure_date < today:
        status = random.choice(["Checked Out", "Cancelled"])
    else:
        status = "Checked In"

    return (
        i,
        random.randint(1, num_hotels),
        random.randint(1, num_room_types),
        10,
        arrival_date.strftime('%Y-%m-%d'),
        departure_date.strftime('%Y-%m-%d'),
        status,
    )

# Create the reservations list here
reservations = [create_reservation(i) for i in range(1, num_reservations + 1)]

def create_payment(i):
    reservation_id = random.randint(1, num_reservations)
    reservation = reservations[reservation_id - 1]
    return (random.randint(1, num_customers), reservation_id, round(random.uniform(50, 500), 2), random.randint(0, 50), reservation[4])

def create_manager(i):
    return (fake.first_name(), fake.last_name(), f'+1({fake.random_number(digits=3):03d})-{fake.random_number(digits=3):03d}-{fake.random_number(digits=4):04d}', random.randint(1, num_hotels))

def create_room_rate(i):
    roomNumber = (i - 1) % num_hotels + 1
    return (random.randint(1, num_room_types), random.randint(1, num_managers), fake.date_between(start_date='-2y', end_date='-1y'), fake.date_between(start_date='-1y', end_date='today'), round(random.uniform(50, 500), 2))


In [ ]:
outputFile = folder_path+"/SQLDataGeneration3.sql"
with open(outputFile, "w+") as f:
    f.write("-- Address\n")
    for i in range(1, num_customers + num_hotels + 1):
        f.write(f"INSERT INTO Address (street, city, state, zipCode) VALUES {create_address(i)};\n")

    f.write("\n-- Membership\n")
    for i in range(1, num_customers + 1):
        f.write(f"INSERT INTO Membership (points) VALUES ({create_membership(i)});\n")

    f.write("\n-- CreditCard\n")
    for i in range(1, num_customers + 1):
        card_token, card_type, expiration_date = create_credit_card(i)
        f.write(f"INSERT INTO CreditCard (cardToken, cardType, expirationDate) VALUES ('{card_token}', '{card_type}', TO_DATE('{expiration_date}', 'YYYY-MM-DD'));\n")

    f.write("\n-- Customer\n")
    for i in range(1, num_customers + 1):
        first_name, last_name, phone_number, card_id, address_id, membership_id = create_customer(i)
        f.write(f"INSERT INTO Customer (firstName, lastName, phoneNumber, cardID, addressID, membershipID) VALUES ('{first_name}', '{last_name}', '{phone_number}',{card_id}, {address_id}, {membership_id});\n")

    f.write("\n-- Hotel\n")
    for i in range(num_customers + 1, num_customers + num_hotels + 1):
        hotel_name, address_id = create_hotel(i)
        f.write(f"INSERT INTO Hotel (hotelName, addressID) VALUES ('{hotel_name}', {address_id});\n")

    f.write("\n-- Amenity\n")
    for hotel_id in range(1, num_hotels + 1):
        num_amenities_for_hotel = random.randint(1, num_amenities)
        selected_amenities = random.sample(range(num_amenities), num_amenities_for_hotel)
        for i in selected_amenities:
            amenity_values = create_amenity(i, hotel_id)
            f.write(f"INSERT INTO Amenity (description, hotelID) VALUES ('{amenity_values[1]}', {amenity_values[2]});\n")

    f.write("\n-- RoomType\n")
    for i in range(1, num_room_types + 1):
        description, max_guests = create_room_type(i)
        f.write(f"INSERT INTO RoomType (description, maxGuests) VALUES ('{description}', {max_guests});\n")

    f.write("\n-- Reservation\n")
    for i in range(1, num_reservations + 1):
        reservation_values = create_reservation(i)
        f.write(f"INSERT INTO Reservation (customerID, hotelID, roomTypeID, numberOfGuests, arrivalDate, departureDate, reservationStatus) VALUES ({', '.join(map(str, reservation_values[:-3]))}, TO_DATE('{reservation_values[-3]}', 'YYYY-MM-DD'), TO_DATE('{reservation_values[-2]}', 'YYYY-MM-DD'), '{reservation_values[-1]}');\n")

    f.write("\n-- Payment\n")
    for i in range(1, num_reservations + 1):
        payment_values = create_payment(i)
        f.write(f"INSERT INTO Payment (customerID, reservationID, amount, pointsUsed, paymentDate) VALUES ({', '.join(map(str, payment_values[:-1]))}, TO_DATE('{payment_values[-1]}', 'YYYY-MM-DD'));\n")

    f.write("\n-- Room\n")
    for i in range(1, num_rooms + 1):
        room_values = create_room(i)
        f.write(f"INSERT INTO Room (hotelID, roomNumber, roomTypeID, roomStatus) VALUES ({room_values[0]}, {room_values[1]}, {room_values[2]}, '{room_values[3]}');\n")

    f.write("\n-- Manager\n")
    for i in range(1, num_managers + 1):
        manager_values = create_manager(i)
        f.write(f"INSERT INTO Manager (firstName, lastName, phoneNumber, hotelID) VALUES ('{manager_values[0]}', '{manager_values[1]}', '{manager_values[2]}', {manager_values[3]});\n")

    f.write("\n-- RoomRate\n")
    for i in range(1, num_room_types * num_managers + 1):
        room_rate_values = create_room_rate(i)
        f.write(f"INSERT INTO RoomRate (roomTypeID, managerID, startDate, endDate, price) VALUES ({room_rate_values[0]}, {room_rate_values[1]}, TO_DATE('{room_rate_values[2]}', 'YYYY-MM-DD'), TO_DATE('{room_rate_values[3]}', 'YYYY-MM-DD'), {room_rate_values[4]});\n")
